<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Get-files" data-toc-modified-id="Get-files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get files</a></span></li><li><span><a href="#Extracting-features" data-toc-modified-id="Extracting-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extracting features</a></span></li><li><span><a href="#Export-DataFrame-to-.CSV" data-toc-modified-id="Export-DataFrame-to-.CSV-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Export DataFrame to .CSV</a></span></li></ul></div>

# Libraries

In [1]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
# For audio playing
import IPython.display as ipd
# For visualizations
import librosa.display
from math import ceil

# Get files 

**To maximize the results obtained from the data and ensure that all samples have the same duration we will use the following function.**

1. Navigate through directories to get files.</li>
2. For each file we get several fragments that overlap the previous and following fragment.</li>


In [64]:
directory = "data"
def gen_audio(window,overlap=0,path=directory):
    for genre in os.listdir(path):
        if os.path.isdir(f"{path}/{genre}"):
            folder=f"{path}/{genre}"
            for song in os.listdir(folder):
                song_path = f"{folder}/{song}"
                if song == ".DS_Store":
                    continue
                y, sr = librosa.load(song_path)
                duration = True
                i=0
                while duration:
                    #print(song_path)
                    song_points = len(y)
                    window_points = window*sr
                    start = int(i*window_points - overlap*window_points)
                    if start<0:
                        start=0
                    end = start + window_points
                    #print(start,end)
                    sample=y[start:end]
                    if end >= song_points:
                        #print("-")
                        sample = y[-window_points:]
                        duration = False
                    i += 1
                    yield genre,sample,sr

In [65]:
audio_generator = gen_audio(60,0.30)

In [66]:
df = pd.DataFrame()

# Extracting features

**The features that are going to be used are:**
- Zero crossings
- Spectral centroid
- Spectral band_width
- Spectral rolloff
- Chroma stft
- Mel Frequency Cepstral Coeﬃcients (mfcc)

In [67]:
%%time
for label, sample, sr in audio_generator:
    y = sample
    zero_crossing = librosa.zero_crossings(y=y, pad=False)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_band_width = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    df = df.append({
        "label":label,
        "sample":sample,
        "sr":sr,
        "zero_crossings": sum(zero_crossing),
        "spectral_centroid": np.mean(spectral_centroid),
        "spectral_band_width": np.mean(spectral_band_width),
        "spectral_rolloff" : np.mean(spectral_rolloff),
        "chroma_stft": [np.mean(c) for c in chroma_stft],
        "mfcc": [np.mean(e) for e in mfcc]
        
    }, ignore_index=True)

CPU times: user 48min 28s, sys: 5min 15s, total: 53min 44s
Wall time: 1h 36min 44s


In [68]:
df.head(1)

,chroma_stft,label,mfcc,sample,spectral_band_width,spectral_centroid,spectral_rolloff,sr,zero_crossings
0,"[0.407141, 0.46533188, 0.3771676, 0.45020363, ...",Supernatural,"[-183.96336, 165.23877, 8.946929, 23.97522, 1....","[3.5482785e-08, 1.0161084e-08, -9.557592e-09, ...",1472.704499,1061.162991,2069.254154,22050.0,60689.0


In [69]:
df[['chroma_stft1','chroma_stft2','chroma_stft3','chroma_stft4','chroma_stft5','chroma_stft6','chroma_stft7','chroma_stft8','chroma_stft9','chroma_stft10','chroma_stft11','chroma_stft12']] = pd.DataFrame(df.chroma_stft.tolist(), index= df.index)
df = df.drop(['chroma_stft'], axis=1)

In [70]:
df[['mfcc1','mfcc2','mfcc3','mfcc4','mfcc5','mfcc6','mfcc7','mfcc8','mfcc9','mfcc10','mfcc11','mfcc12','mfcc13','mfcc14','mfcc15','mfcc16','mfcc17','mfcc18','mfcc19','mfcc20']] = pd.DataFrame(df.mfcc.tolist(), index= df.index)
df = df.drop(['mfcc'], axis=1)

In [71]:
df.head(1)

,label,sample,spectral_band_width,spectral_centroid,spectral_rolloff,sr,zero_crossings,chroma_stft1,chroma_stft2,chroma_stft3,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,Supernatural,"[3.5482785e-08, 1.0161084e-08, -9.557592e-09, ...",1472.704499,1061.162991,2069.254154,22050.0,60689.0,0.407141,0.465332,0.377168,...,5.959533,0.46896,-0.010982,2.131289,0.01979,-2.533675,-4.513143,-0.910481,-5.09341,-3.961309


In [72]:
df.shape

(553, 39)

# Export DataFrame to .CSV

In [73]:
df.to_csv(path_or_buf= "output/new_features_60s_030ov.csv",index=False)